In [1]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
#Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size = 1500)
lngs = np.random.uniform(-90, 90, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs = list(map(list, lat_lngs))
lat_lngs

[[49.927617345843004, 49.66131489555772],
 [-20.638842355115713, -77.95618926086586],
 [-33.27578394354491, -31.045740000996524],
 [-1.1818725684893252, 48.26830915204104],
 [17.060395788108636, 27.908059928532552],
 [-40.820627863486955, 83.19828439052412],
 [43.453409134406, 34.229964810444656],
 [44.85977738013128, 29.124786201620992],
 [68.15770930641864, 69.96004878298265],
 [77.94800997211627, -71.69711504040916],
 [-68.4197577888519, -82.61497298812716],
 [-33.29793546548854, -87.0004706442499],
 [21.1213282885589, -32.75930663193047],
 [-50.19667912136399, 18.56087200058998],
 [-84.98016366770827, -27.355182331944455],
 [-76.05724422712157, 82.06367317375725],
 [63.235957805567324, 50.34258789171571],
 [28.913495633526665, -75.82324260927834],
 [-20.948391761572807, -27.3635795914197],
 [22.125120967821218, -82.73043857997705],
 [29.94284657361021, -53.214316825343495],
 [6.531769525808258, -10.94367155995819],
 [-2.247803981945893, -41.22491712839313],
 [75.45741186216819, -22

In [3]:
#Create a list for holding the cities
cities = []

#Identify the nearest city for each lat and long combination
for coordinate in lat_lngs:
    city = citipy.nearest_city(lat_lngs[0], lat_lngs[1]).city_name
    
    #If the city is unique, then add it to the list
    if city not in cities:
        cities.append(city)

len(cities)

TypeError: unsupported operand type(s) for -: 'float' and 'list'